# Quickprop, an Alternative to Back-Propagation

#### Scott Fahlman's idea to speed up gradient descent

Due to the slowly converging nature of the vanilla back-propagation algorithms of the 80's/90's, Scott Fahlman invented a learning algorithm dubbed Quickprop [1] that is roughly leaning on [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method).
His simple idea outperformed back-propagation (with various adjustments) on problem domains like the 'N-M-N Encoder' task - i.e. training an de/encoder network with N inputs, M hidden units and N outputs.  
One of the problems that Quickprop specifically tackles is the issue of finding a domain-specific optimal learning rate, or rather: an algorithm that adjusts it appropriately dynamically.

In in this article, we'll look at the simple mathematical idea behind Quickprop.
We'll implement the basic algorithm and some improvements that Fahlman suggests - all in Python and PyTorch.

A rough implementation of the algorithm and some background can already be found in [this useful blog post](https://www.bonaccorso.eu/2017/09/15/quickprop-an-almost-forgotten-neural-training-algorithm/) by Giuseppe Bonaccorso. We are going to expand on that - both on the theory and code side - but if in doubt, have a look at how Giuseppe explains it.

---

_The motivation to look into Quickprop came from writing [my last article](https://towardsdatascience.com/cascade-correlation-a-forgotten-learning-architecture-a2354a0bec92) on the "Cascade-Correlation Learning Architecture" [2]. There, I used it to train the neural network's output and hidden neurons, which was a mistake I realized only later and which we'll also look into here._

_To follow along with this article, you should be familiar with how neural networks can be trained using back-propagation of the loss gradient (as of 2020, a widely used approach). That is, you should understand how the gradient is usually calculated and applied to the parameters of a network to try to iteratively achieve convergence of the loss to a global minimum._

---

## Overview

We'll start with the mathematics behind Quickprop and then look at how it can be implemented and improved step by step.  
To make following along easier, any equations used and inference steps done are explained in more detail than in the original paper.

## The Mathematics Behind Quickprop

The often used learning method of back-propagation for neural networks is based on the idea of iteratively 'riding down' the slop of a function, by taking short steps in the inverse direction of its gradient.

These 'short steps' are the crux here. Their length usually depends on a learning rate factor, and that is kept intentionally small to not overshoot a potential minimum.

Back in the days when Fahlman developed Quickprop, choosing a good learning rate was something of a major problem. As he actually mentions in his paper, in the best performing algorithm, the scientist chose the learning rate 'by eye' (i.e. manually and based on experience) every step along the way! [1]

Faced with this, Fahlman came up with a different idea: Solving a simpler problem.

Minimizing the loss function ***L***, especially for deep neural networks, can become extremely difficult analytically (i.e. in a general way on the entire domain).  
In back-propagation, for instance, we only calculate it point-wise and then do the small steps in the right direction. If we would know how the 'terrain' of the function looks like in general, we could 'jump' to the minimum directly.

But what if we could replace the loss function with a simpler version, of which we know its terrain?
This is exactly Fahlmans' assumption taken in Quickprop: He presumes that ***L*** can be approximated by a simple parabola that opens in the positive direction. This way, calculating the minimum (of the parabola) is as simple as finding the intersection of a line with the x-axis.

And if that point is not yet a minimum of the loss function, the next parabola can be approximated from there, like in the graphic below.

![Animation of Quickprop](./img/quickprop.gif)

#### A parabola is fit to the original function and a step is taken towards its minimum. From there, the next parabola is fit and the next step is taken. The two dotted lines are the current and a previous stationary point of the parabola. (Graphic by author)

So... How exactly can we approximate ***L***? Easy - using a [Taylor series](https://en.wikipedia.org/wiki/Taylor_series), and a small trick.

_Note that for the following equations, we consider the components of the weight vector ***w*** to be trained independently, so ***w*** is meant to be seen as a scalar. But we can still exploit the SIMD architecture of GPU's, using component-wise computations._

We start off with the second order Taylor expansion of ***L***, giving us a parabola (without an error term):

$$
\begin{align}
T(L, w_{n+1}) &= L(w_n) + \frac{L'(w_n)}{1!} (w_{n+1} - w_n) + \frac{L''(w_n)}{2!} (w_{n+1} - w_n)^{2}\\
\end{align} 
$$

(To understand how this was created, check out the Wikipedia article on Taylor series linked above - it's as simple as inputting ***L*** into the general Taylor formula up to the second term and dropping the rest.)

We can now define the update rule for the weights based on a weight difference, and input that into ***T***:

$$
\begin{align}
&& w_{n+1} &= w_n + \Delta w_n \\
\implies && T(L, w_n + \Delta w_n) &= L(w_n) + \frac{L'(w_n)}{1!} (\Delta w_n) + \frac{L''(w_n)}{2!} (\Delta w_n)^{2}\\
\end{align} 
$$

Quickprop now further approximates ***L''*** linearly using the difference quotient (this is the small trick mentioned above):

$$
\begin{align}
L''(w_n) &\approx \frac{L'(w_n) - L'(w_{n-1})}{\Delta w_{n-1}} \\
\end{align} 
$$

Using this, we can rewrite the Taylor polynomial to this 'Quickprop' adjusted version and build its gradient:

$$
\begin{align}
T_Q(L, w_n + \Delta w_n) &= L(w_n) + L'(w_n) (\Delta w_n) + 1/2 \ \frac{L'(w_n) - L'(w_{n-1})}{\Delta w_{n-1}} (\Delta w_n)^{2} \\
T_Q'(L, w_n + \Delta w_n) &= L'(w_n) + \frac{L'(w_n) - L'(w_{n-1})}{\Delta w_{n-1}} \Delta w_n \\
\end{align} 
$$

And that last equation, finally, can be used to calculate the stationary point of the parabola:

$$
\begin{align}
&& L'(w_n) + \frac{L'(w_n) - L'(w_{n-1})}{\Delta w_{n-1}} \Delta w_n & = 0 \\
\Leftrightarrow && \frac{L'(w_n) - L'(w_{n-1})}{\Delta w_{n-1}} \Delta w_n &= -L'(w_n) \\
\Leftrightarrow && \Delta w_n &= \Delta w_{n-1} \frac{-L'(w_n)}{L'(w_n) - L'(w_{n-1})}  \\
\Leftrightarrow && \Delta w_n &= \Delta w_{n-1} \frac{L'(w_n)}{L'(w_{n-1}) - L'(w_n)}  \\
\end{align} 
$$

**That's it!** Now, to put things together, given a previous weight, a previous weight difference and the loss slope at the previous and current weight, Quickprop calculates the new weight simply by:

$$
\begin{align}
\Delta w_n &= \Delta w_{n-1} \frac{L'(w_n)}{L'(w_{n-1}) - L'(w_n)}  \\
w_{n+1} &= w_n + \Delta w_n \\
\end{align} 
$$

## Put It Into Code

Before starting with the actual Quickprop implementation, let's import some foundational libraries:

In [ ]:
import numpy as np
import torch

With the last two lines of the mathematical equation from earlier, we can start with Quickprop! If you read the first article on Cascade-Correlation, you might be already familiar with this - here, we'll concentrate on essential parts of the algorithm first, and put it all together in the end.

_Note that we use PyTorch to do the automatic gradient calculation for us. We also assume to have defined an activation and loss function beforehand._

In [ ]:
# Setup torch autograd for weight vector w
w_var = torch.autograd.Variable(torch.Tensor(w), requires_grad=True)

# Calc predicted values based on input x and loss based on expected output y
predicted = activation(torch.mm(x, w_var))
L = loss(predicted, y)

# Calc differential
L.backward()

# And, finally, do the weight update
dL = w_var.grad.detach() # =: partial(L) / partial(W)
dw = dw_prev * dL / (dL_prev - dL)

dw_prev = dw.clone()

w += learning_rate * dw

This is the simplest Quickprop version for one epoch of learning. To actually make use of it, we'll have to run it several times and see if the loss converges (we'll cover that bit later).

However, this implementation is flawed in several ways, which we are going to investigate and fix in the following sections:

- We didn't actually initialize any of the `..._prev` variables - in the last article I statically initialized them with ones, but that is also not a good idea (see next points)
- The weight delta variable might get stuck on zero values, since it is used as a factor in its own update step
- The implementation might overshoot or generally fail to converge, if the gradient 'explodes'
- It will result in division by zero if the gradient doesn't change in one iteration

## Improvement: Init via Gradient Descent

The first simple fix we can apply is using gradient descent (with a very small learning rate) to prepare the `dw_prev` and `dL_prev` variables. This will give us a good first glimpse of the loss function terrain, and kick-starts Quickprop in the right direction.

Gradient descent is easily implemented using pytorch again - we'll also use the opportunity to refactor the code above a bit as well:

In [ ]:
def calc_gradient(x, y, w, activation, loss):
    # Helper to calc loss gradient
    w_var = torch.autograd.Variable(torch.Tensor(w), requires_grad=True)
    predicted = activation(torch.mm(x, w_var))
    L = loss(predicted, y)
    L.backward()
    dL = w_var.grad.detach()
    return L, dL, predicted
    
def grad_descent_step(x, y, w, activation, loss, learning_rate=1e-5):
    # Calculate the gradient as usually
    L, dL, predicted = calc_gradient(x, y, w, activation, loss)
    
    # Then do a simple gradient descent step
    dw = -learning_rate * dL
    new_w = w + dw
    
    return new_w, dw, L, dL

## Improvement: Conditional Gradient Addition

Sometimes, the weight deltas become vanishingly small when using the Quickprop parabola approach. To prevent that from happening when the gradient is not zero, Fahlman recommends conditionally adding the slope to the weight delta.  
The idea can be described like this: Go further if you have been moving in that direction anyway, but don't push on if your previous update sent you in the opposite direction (to prevent oscillation).

With a little piece of decider code, this can be implemented quite easily:

In [ ]:
# (This code is just to illustrate the process before the real implementation, it won't execute)

# We'll receive dw and dw_prev and need to decide whether to apply the update or not.
# To not have to include conditional execution (if clauses) we'll want to do it branchless.
# This can be achieved by a simple mutliplication rule using the sign function:

# Sign gives us either -1, 0 or 1 based on the parameter being less, more or exactly zero
# (check the docs for specifics),
    np.sign(dw) + np.sign(dw_prev)
# With this, we'll have three cases as the outcome of the sum to consider here:
# -2, -1, 0, 1, 2
# But actually, we're really only interested if this is 0 or not, so we can do:
    np.clip(np.abs(np.sign(dw) + np.sign(dw_prev)), a_min=0, a_max=1)
# And use that as our deciding factor, which is either 1 or 0 when the dw and dw_prev share the sign or not.

With this, we can put it all into one small function:

In [ ]:
def cond_add_slope(dw, dw_prev, dL, learning_rate=1.5):
    ddw = np.clip(np.abs(np.sign(dw) + np.sign(dw_prev)), a_min=0, a_max=1)
    return dw + ddw * (-learning_rate * dL)

## Improvement: Maximum Growth Factor

As a second step, we'll fix the issue of exploding weight deltas near some function features (e.g. near singularities).  
To do that, Fahlman suggests to clip the weight update, if it would be bigger than the last weight update times a maximum grow factor:

In [ ]:
def clip_at_max_growth(dw, dw_prev, max_growth_factor=1.75):
    # Get the absolute maximum element-wise growth
    max_growth = max_growth_factor * np.abs(dw_prev)
    
    # And implement this branchless with a min/max clip
    return np.clip(dw, a_min=(-max_growth), a_max=max_growth)

## Improvement: Prevent Division by Zero

On some occasions, the previous and current computed slope can be the same. The result is that we'll try to divide by zero in the weight update rule, and will afterward continue having to work with `NaN`'s, which obviously breaks the training.  
The simple fix here is to do a gradient descent step instead.

Observe the two update rules:

In [ ]:
# Quickprop
dw = dw_prev * dL / (dL_prev - dL)
# Gradient descent
dw = -learning_rate * dL

# We'll get a nicer result if we shuffle the equations a bit:
dw = dL * dw_prev / (dL_prev - dL)
dw = dL * (-learning_rate)

Besides the last factor, they look similar, no?  
Which means we can go branchless again (i.e. save us some if-clauses), stay element-wise and pack everything in one formula:

In [ ]:
# (This code is just to illustrate the process before the real implementation, it won't execute)

# If (dL_prev - dL) is zero, we want to multiply the learning rate instead,
# i.e. we want to switch to gradient descent. We can accomplish it this way:

# First, we make sure we only use absolute values (the 'magnitude', but element-wise)
    np.abs(dL_prev - dL)
# Then we map this value onto either 0 or 1, depending on if it is 0 or not (using the sign function)
ddL = np.sign(np.abs(dL_prev - dL))

# We can now use this factor to 'decide' between quickprop and gradient descent:
quickprop_factor = ddL       * (dw_prev / (dL_prev - dL))
grad_desc_factor = (1 - ddL) * (-learning_rate)

# Overall we get:
dw = dL * (quickprop_factor + grad_desc_factor)

The attentive reader probably noted the 'learning rate' factor we used above - a parameter we thought we could get rid of...  
Well, actually we sort of did, or at least we did get rid of the problem of having to adjust the learning rate over the course of the training.
The Quickprop learning rate can stay fixed throughout the process.
It only has to be adjusted once per domain in the beginning.
The actual dynamic step sizes are chosen through the parabola jumps, which in turn depend heavily on the current and last calculated slope.

If you think this sounds awfully familiar to how back-propagation learning rate optimizers work (think: momentum), you'd be on the right track. In essence, Quickprop achieves something very similar to them - just that it doesn't use back-propagation at its core.

Coming back to the code: Since we already implemented gradient descent earlier on, we can build on that and re-use as much as possible:

In [ ]:
def quickprop_step(x, y, w, dw_prev, dL_prev,
                   activation, loss,
                   qp_learning_rate=1.5,
                   gd_learning_rate=1e-5):
    # Calculate the gradient as usually
    L, dL, predicted = calc_gradient(x, y, w, activation, loss)
    
    # Calculate a 'decider' bit between quickprop and gradient descent
    ddL = np.ceil(np.clip(np.abs(dL_prev - dL), a_min=0, a_max=1) / 2)
    
    quickprop_factor = ddL       * (dw_prev / (dL_prev - dL))
    grad_desc_factor = (1 - ddL) * (-gd_learning_rate)

    dw = dL * (quickprop_factor + grad_desc_factor)
    
    # Use the conditional slope addition
    dw = cond_add_slope(dw, dw_prev, dL, qp_learning_rate)
    
    # Use the max growth factor
    dw = clip_at_max_growth(dw, dw_prev)

    new_w = w + dw
    
    return new_w, dw, L, dL, predicted

## Putting It All Together

With all of these functions in place, we can put it all together.
The bit of boilerplate code still necessary just does the initialization and checks for convergence of the mean loss per epoch.

In [ ]:
# Param shapes: x_: (n,i), y_: (n,o), weights: (i,o)
#   Where n is the size of the whole sample set, i is the input count, o is the output count
#   We expect x_ to already include the bias
# Returns: trained weights, last prediction, last iteration, last loss
# NB: Differentiation is done via torch
def quickprop(x_, y_, weights,
              activation=torch.nn.Sigmoid(),
              loss=torch.nn.MSELoss(),
              learning_rate=1e-4,
              tolerance=1e-6,
              patience=20000,
              debug=False):
    # Box params as torch datatypes
    x = torch.Tensor(x_)
    y = torch.Tensor(y_)
    w = torch.Tensor(weights)

    # Keep track of mean residual error values (used to test for convergence)
    L_mean = 1
    L_mean_prev = 1
    L_mean_diff = 1
    
    # Keep track of loss and weight gradients
    dL = torch.zeros(w.shape)
    dL_prev = torch.ones(w.shape)
    dw_prev = torch.ones(w.shape)
    
    # Initialize the algorithm with a GD step
    w, dw_prev, L, dL_prev = grad_descent_step(x, y, w, activation, loss)

    i = 0
    predicted = []

    # This algorithm expects the mean losses to converge or the patience to run out...
    while L_mean_diff > tolerance and i < patience:
        # Prep iteration
        i += 1
        dL_prev = dL.clone()
        
        w, dw, L, dL, predicted = quickprop_step(x, y, w, dw_prev, dL_prev, activation, loss, qp_learning_rate=learning_rate)
        
        dw_prev = dw.clone()
        
        # Keep track of losses and use as convergence criterion if mean doesn't change much     
        L_mean = L_mean + (1/(i+1))*(L.detach().numpy() - L_mean)
        L_mean_diff = np.abs(L_mean_prev - L_mean)
        L_mean_prev = L_mean
        
        if debug and i % 100 == 99:
            print("Residual           ", L.detach().numpy())
            print("Residual mean      ", L_mean)
            print("Residual mean diff ", L_mean_diff)
        
    return w.detach().numpy(), predicted.detach().numpy(), i, L.detach().numpy()

## Caveats

Quickprop has one major caveat that greatly reduces its usefulness: The mathematical 'trick' we used, i.e. the approximation of the second order derivative of the loss function with a simple difference quotient relies on the assumption that this second order derivative is a continuous function.  
This is not given for activation functions like e.g. the rectified linear unit, or ReLU for short. The second-order derivative is discontinuous and the behaviour of the algorithm might become unreliable (e.g. it might diverge).

Looking back at [my earlier article](https://towardsdatascience.com/cascade-correlation-a-forgotten-learning-architecture-a2354a0bec92) covering the implementation of Cascade-Correlation, we trained the hidden units of the network using Quickprop and used the covariance function as a way to estimate loss in that process.
However, the covariance (as implemented there) is wrapped in an absolute value function.
I.e. its second-order derivative is discontinuous and therefore, Quickprop should not be used.
The careful reader of Fahlman et al.'s Cascade-Correlation paper [2] may have also noticed that they are actually using gradient ascent to calculate this maximum covariance.

Apart from that, it also seems that Quickprop delivers better results on some domains rather than others. An interesting summary by Brust et al. showed that it achieved better training results compared to the quality of back-propagation based techniques on some simple image classification tasks (classifying basic shapes) while at the same time doing worse on more realistic image classification tasks [3].  
I haven't done any research in that direction, but I wonder if this could imply that Quickprop might work better on less fuzzy and more structured data (think data frames/tables used in a business context). That would surely be interesting to investigate.

## Summary

This article covered Scott Fahlman's idea of improving back-propagation. We had a look at the mathematical foundations and a possible implementation.

Now go about and try it out for your own projects - I'd love to see what Quickprop can used for!

---

If you would like to see variants of Quickprop in action, check out [my series of articles](https://towardsdatascience.com/cascade-correlation-a-forgotten-learning-architecture-a2354a0bec92) on the Cascade-Correlation Learning Architecture.

All finished notebooks and code of this series are also [available on Github](https://github.com/ephe-meral/cascor). Please feel encouraged to leave feedback and suggest improvements.

---

[1] S. E. Fahlman, [An empirical study of learning speed in back-propagation networks](http://www.it.uu.se/edu/course/homepage/mil/vt11/handouts/fahlman.quickprop-tr.pdf) (1988), Carnegie Mellon University, Computer Science Department

[2] S. E. Fahlman and C. Lebiere, [The cascade-correlation learning architecture](http://web.cs.iastate.edu/~honavar/fahlman.pdf) (1990), Advances in neural information processing systems (pp. 524–532)

[3] C. A. Brust, S. Sickert, M. Simon, E. Rodner and J. Denzler, [Neither Quick Nor Proper - Evaluation of QuickProp for Learning Deep Neural Networks](https://arxiv.org/pdf/1606.04333.pdf) (2016), arXiv preprint arXiv:1606.04333